In [6]:
import numpy as np 
import matplotlib.pyplot as plt 
import sympy as sp 

#defining numerical error for basis function calculations
eps = 0



In [7]:
#This cell contains sympy working for generating derivatives of the Galerkin basis function
#all sympy variables have prefix sp_

sp_x,sp_z,sp_k,sp_x_j,sp_z_j, sp_H, sp_L = sp.symbols('sp_x sp_z sp_k sp_x_j sp_z_j sp_H sp_L')

sp_r = (sp.sqrt((sp_x-sp_x_j)**2 + (sp_z-sp_z_j)**2)) + eps #radial distance from X to X_j
sp_phi = sp_r**sp_k*sp.log(sp_r+eps) #basis function




#ensure basis function is zero on boundaries
sp_bFunc = sp_x*sp_z*(sp_x-sp_L)*(sp_z-sp_H) 

#Boundary condition is approx 1 on z=H and zero everywhere else
sp_sigma = 0.1
sp_bc_func = sp.exp(-0.5*(sp_H-sp_z)**2/sp_sigma**2) 


#total basis function with boundary conditions
sp_phi_tot = sp_bFunc*sp_phi
#sp_phi_tot = sp.simplify(sp_phi_tot)




#finding Laplacian of basis function
sp_nabla2phi = sp_phi_tot.diff(sp_x,2) + sp_phi_tot.diff(sp_z,2)
#sp_nabla2phi = sp.simplify(sp_nabla2phi)

#finding Laplacian of boundary condition function
sp_nabla2bc = sp_bc_func.diff(sp_x,2) + sp_bc_func.diff(sp_z,2)
#sp_nabla2bc = sp.simplify(sp_nabla2bc)

#finding x gradients for testing purposes
sp_xgrad = sp_phi_tot.diff(sp_x)

#lambdifying expressions for use in numpy

nabla2phi = sp.lambdify((sp_x,sp_z,sp_x_j,sp_z_j,sp_k,sp_L,sp_H),sp_nabla2phi )
nabla2bc =  sp.lambdify((sp_x,sp_z,sp_x_j,sp_z_j,sp_k,sp_L,sp_H),sp_nabla2bc  )
phitot = sp.lambdify((sp_x,sp_z,sp_x_j,sp_z_j,sp_k,sp_L,sp_H),sp_phi_tot  )
xgrad = sp.lambdify((sp_x,sp_z,sp_x_j,sp_z_j,sp_k,sp_L,sp_H),sp_xgrad  )
bctot = sp.lambdify((sp_x,sp_z,sp_x_j,sp_z_j,sp_k,sp_L,sp_H),sp_bc_func )


In [8]:
#defining state variables:
H = 1
L = 2
dx = 0.1 #grid points
k = 4
xx,zz = np.meshgrid(np.arange(0,L+dx,dx),np.arange(0,H+dx,dx))

test = False
#testing and plotting laplacian
if test:
    x_j = 1
    z_j = 0.5
    phitot_test = phitot(xx,zz,x_j,z_j,k,L,H)
    xgrad_test = xgrad(xx,zz,x_j,z_j,k,L,H)
    nabla2phi_test = nabla2phi(xx,zz,x_j,z_j,k,L,H)
    plt.plot(phitot_test[5,:])
    xgrad_numerical = (phitot_test[:,1:] - phitot_test[:,0:-1])/dx
    plt.plot(xgrad_numerical[5,:]/10,'.')
    plt.plot(xgrad_test[5,:]/10)



In [9]:
#setting up solution of Laplace's equation
Npoints = xx.shape[0]*xx.shape[1]
xp = xx.reshape((Npoints,1))
zp = zz.reshape((Npoints,1))
eps = 1e-6

#building phi matrix
nabPHI = np.zeros((Npoints,Npoints))
pHI = np.zeros((Npoints,Npoints))

#populating matrices
for j in range(0,Npoints):
    for x in range(0,Npoints):

        #building matrix values
        nabPHI_val = nabla2phi(xp[x,0],zp[x,0],xp[j,0]+eps,zp[j,0]+eps,k,L,H)
        pHI_val = phitot(xp[x,0],zp[x,0],xp[j,0]+eps,zp[j,0]+eps,k,L,H)

        #checking for Nans
        if np.isnan(pHI_val):
            pHI_val = 0

        if np.isnan(nabPHI_val):
            nabPHI_val = 0

        #constructing matrices
        nabPHI[x,j] = nabPHI_val
        pHI[x,j] = pHI_val

#building boundary condition matrix
bc_vector = np.zeros((Npoints,1))
nabbc_vector = np.zeros((Npoints,1))

for x in range(0,Npoints):
    bc_val = nabla2bc(xp[x,0],zp[x,0],xp[0,0],zp[0,0],k,L,H)
    nabbc_val = bctot(xp[x,0],zp[x,0],xp[0,0],zp[0,0],k,L,H)

    #checking for nans
    if np.isnan(bc_val):
        bc_val = 0

    if np.isnan(nabbc_val):
        nabbc_val = 0

    bc_vector[x] = bc_val
    nabbc_val = nabbc_vector



In [10]:
#solving equation:

u_k = np.linalg.solve(nabPHI,-nabbc_vector)

LinAlgError: Singular matrix